In [1]:
import pandas as pd
import folium

In [2]:
def generateBaseMap(default_location=[40.693943, -73.985880], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [3]:
basemap = generateBaseMap()
basemap

In [4]:
raw_data_copy = pd.read_csv(r'pollution_us_2000_2016.csv')
raw_data_copy['State County Site Code'] = raw_data_copy['State Code'].astype(str) + '-' + raw_data_copy['County Code'].astype(str) + '-' + raw_data_copy['Site Num'].astype(str)
raw_data_copy['SO2 AQI'] = raw_data_copy['SO2 AQI'].fillna(raw_data_copy.groupby(['State County Site Code', 'Date Local'])['SO2 AQI'].transform('mean'))
raw_data_copy['CO AQI'] = raw_data_copy['CO AQI'].fillna(raw_data_copy.groupby(['State County Site Code', 'Date Local'])['CO AQI'].transform('mean'))
raw_data_copy = raw_data_copy.drop(['Unnamed: 0', 'State Code', 'County Code', 'Site Num'], axis = 1)
new_data = raw_data_copy.groupby(['Address', 'Date Local'])[['NO2 Mean','NO2 AQI','O3 Mean','O3 AQI','SO2 Mean','SO2 AQI','CO Mean','CO AQI',]].transform('mean')
new_data['Address'] = raw_data_copy['Address']
new_data['State'] = raw_data_copy['State']
new_data['County'] = raw_data_copy['County']
new_data['City'] = raw_data_copy['City']
new_data['Date Local'] = raw_data_copy['Date Local']
new_data['NO2 Units'] = raw_data_copy['NO2 Units']
new_data['O3 Units'] = raw_data_copy['O3 Units']
new_data['SO2 Units'] = raw_data_copy['SO2 Units']
new_data['CO Units'] = raw_data_copy['CO Units']
raw_data_copy = new_data
data_with_site = raw_data_copy.drop_duplicates()
new_data = data_with_site.groupby(['City', 'Date Local'])[['NO2 Mean','NO2 AQI','O3 Mean','O3 AQI','SO2 Mean','SO2 AQI','CO Mean','CO AQI']].transform('mean')
new_data['State'] = raw_data_copy['State']
new_data['County'] = raw_data_copy['County']
new_data['City'] = raw_data_copy['City']
new_data['Date Local'] = raw_data_copy['Date Local']
new_data['NO2 Units'] = raw_data_copy['NO2 Units']
new_data['O3 Units'] = raw_data_copy['O3 Units']
new_data['SO2 Units'] = raw_data_copy['SO2 Units']
new_data['CO Units'] = raw_data_copy['CO Units']
data_with_city = new_data
data_with_city = data_with_city.drop_duplicates()

In [5]:
data_with_city.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392352 entries, 0 to 1746657
Data columns (total 16 columns):
NO2 Mean      392352 non-null float64
NO2 AQI       392352 non-null float64
O3 Mean       392352 non-null float64
O3 AQI        392352 non-null float64
SO2 Mean      392352 non-null float64
SO2 AQI       392352 non-null float64
CO Mean       392352 non-null float64
CO AQI        392178 non-null float64
State         392352 non-null object
County        392352 non-null object
City          392352 non-null object
Date Local    392352 non-null object
NO2 Units     392352 non-null object
O3 Units      392352 non-null object
SO2 Units     392352 non-null object
CO Units      392352 non-null object
dtypes: float64(8), object(8)
memory usage: 50.9+ MB


In [6]:
data_with_city['Date Local'] = pd.to_datetime(data_with_city['Date Local'])
data_with_city.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392352 entries, 0 to 1746657
Data columns (total 16 columns):
NO2 Mean      392352 non-null float64
NO2 AQI       392352 non-null float64
O3 Mean       392352 non-null float64
O3 AQI        392352 non-null float64
SO2 Mean      392352 non-null float64
SO2 AQI       392352 non-null float64
CO Mean       392352 non-null float64
CO AQI        392178 non-null float64
State         392352 non-null object
County        392352 non-null object
City          392352 non-null object
Date Local    392352 non-null datetime64[ns]
NO2 Units     392352 non-null object
O3 Units      392352 non-null object
SO2 Units     392352 non-null object
CO Units      392352 non-null object
dtypes: datetime64[ns](1), float64(8), object(7)
memory usage: 50.9+ MB


C:\Users\shubh\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
cols_to_include = ['city', 'county_name', 'state_name', 'lat', 'lng']
coordinates = pd.read_csv('uscities.csv')[cols_to_include]
combined = pd.merge(data_with_city, coordinates, how = 'left', 
                   left_on = ['City', 'County', 'State'], right_on = ['city', 'county_name', 'state_name']).drop(['city', 'county_name', 'state_name'], axis = 1)

In [8]:
from folium.plugins import HeatMap

In [12]:
basemap = generateBaseMap()

In [21]:
HeatMap(data= combined[['lat', 'lng', 'NO2 Mean']].head(100).dropna().values.tolist()).add_to(basemap)

In [18]:
# map_2 = folium.Map(location=[45.372, -121.6972],
# zoom_start=12,
# tiles='Stamen Terrain')
# folium.Marker([45.3288, -121.6625]).add_to(map_2)
# folium.Marker([45.3311, -121.7113]).add_to(map_2)
# map_2

In [11]:
# cols_to_include = ['city', 'county_name', 'state_name', 'lat', 'lng']
# coordinates = pd.read_csv('uscities.csv')[cols_to_include]
# combined = pd.merge(pollution_df, coordinates, how = 'left', 
#                    left_on = ['City', 'County', 'State'], right_on = ['city', 'county_name', 'state_name']).drop(['city', 'county_name', 'state_name'], axis = 1)